<a href="https://www.kaggle.com/code/adends/titanic-competition-attempt-1?scriptVersionId=164639461" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import re

In [3]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [4]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
numerical = ['PassengerId','Pclass','SibSp','Parch','Fare', 'Age']
categorical = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [7]:
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=True)
df_test['Title'] = df_test['Name'].str.extract('([A-Za-z]+)\.', expand=True)

print(df[['Name', 'Title']])

                                                  Name Title
0                              Braund, Mr. Owen Harris    Mr
1    Cumings, Mrs. John Bradley (Florence Briggs Th...   Mrs
2                               Heikkinen, Miss. Laina  Miss
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)   Mrs
4                             Allen, Mr. William Henry    Mr
..                                                 ...   ...
886                              Montvila, Rev. Juozas   Rev
887                       Graham, Miss. Margaret Edith  Miss
888           Johnston, Miss. Catherine Helen "Carrie"  Miss
889                              Behr, Mr. Karl Howell    Mr
890                                Dooley, Mr. Patrick    Mr

[891 rows x 2 columns]


In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [9]:
df.drop(columns=['PassengerId', 'Name', 'Ticket'],axis=1,inplace=True)
df_test.drop(columns=['PassengerId', 'Name', 'Ticket'],axis=1,inplace=True)

In [10]:
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(df[['Sex', 'Embarked', 'Cabin', 'Title']])
encoded_data = encoder.transform(df[['Sex', 'Embarked', 'Cabin', 'Title']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Sex', 'Embarked', 'Cabin', 'Title']))
df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
df = df.drop(['Sex', 'Embarked', 'Cabin', 'Title'], axis=1)

encoder.fit(df_test[['Sex', 'Embarked', 'Cabin', 'Title']])
encoded_data_test = encoder.transform(df_test[['Sex', 'Embarked', 'Cabin', 'Title']])
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoder.get_feature_names_out(['Sex', 'Embarked', 'Cabin', 'Title']))
df_test = pd.concat([df_test.reset_index(drop=True), encoded_df_test.reset_index(drop=True)], axis=1)
df_test = df_test.drop(['Sex', 'Embarked', 'Cabin', 'Title'], axis=1)

df

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
887,1,1,19.0,0,0,30.0000,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0,3,NaN,1,2,23.4500,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,1,1,26.0,0,0,30.0000,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
df.isna().sum()

Survived       0
Pclass         0
Age          177
SibSp          0
Parch          0
            ... 
Title_Mr       0
Title_Mrs      0
Title_Ms       0
Title_Rev      0
Title_Sir      0
Length: 177, dtype: int64

In [12]:
scaler = MinMaxScaler()
scaler.fit(df)
scaled_df = scaler.transform(df)
df = pd.DataFrame(scaled_df, columns=df.columns)

scaler.fit(df_test)
scaled_df_test = scaler.transform(df_test)
df_test = pd.DataFrame(scaled_df_test, columns=df_test.columns)

In [13]:
imputer = IterativeImputer()
imputer.fit(df)
imputed_df = imputer.transform(df)
df = pd.DataFrame(imputed_df, columns=df.columns)

imputer.fit(df_test)
imputed_df_test = imputer.transform(df_test)
df_test = pd.DataFrame(imputed_df_test, columns=df_test.columns)

In [14]:
df.isna().sum()

Survived     0
Pclass       0
Age          0
SibSp        0
Parch        0
            ..
Title_Mr     0
Title_Mrs    0
Title_Ms     0
Title_Rev    0
Title_Sir    0
Length: 177, dtype: int64

In [15]:
feature_df = df.drop('Survived', axis=1)

pca = PCA(n_components=0.95)

df_pca = pca.fit_transform(feature_df)

df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(df_pca.shape[1])])

df_pca


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67
0,-0.786873,-0.379028,-0.024495,0.024325,-0.081346,-0.056198,0.118048,0.044786,-0.036042,0.082595,...,0.005645,-0.008040,0.002748,-0.002023,0.005302,-0.005744,0.004200,0.000795,-0.001185,2.352138e-16
1,1.376854,1.297806,0.169032,0.743463,0.018718,-0.021555,0.136789,-0.005168,-0.085414,0.085678,...,0.016384,-0.002193,-0.031375,0.083654,0.031337,-0.054118,0.097445,0.009473,0.019150,1.629530e-01
2,1.088202,-0.790201,-0.137384,-0.432030,-0.234221,-0.145451,0.049932,-0.188335,0.010903,-0.035821,...,-0.004560,0.011411,-0.011452,-0.012044,-0.000772,-0.001176,-0.003285,0.000246,-0.002005,-1.465405e-16
3,1.188027,0.308348,1.098282,0.521055,0.312299,-0.010609,0.109696,0.015853,-0.151469,0.111025,...,-0.078396,-0.002801,-0.132358,-0.055710,-0.035934,0.081965,-0.091426,-0.003639,-0.001528,1.844044e-16
4,-0.793863,-0.358415,-0.008569,0.035729,-0.053330,-0.121897,0.083334,-0.085945,0.031957,-0.024824,...,0.006858,0.005456,-0.001617,-0.008544,0.002250,-0.000314,-0.002954,0.000155,-0.001581,-1.431419e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.228365,-0.228417,0.230966,0.017662,-0.073481,0.465690,-0.472531,-0.438200,0.209942,0.385695,...,-0.003130,-0.015115,-0.003443,-0.001136,0.026351,-0.003628,0.012559,-0.029379,0.012502,-3.149128e-17
887,1.308512,0.156486,0.742953,-0.818269,-0.000027,-0.035741,0.066446,-0.053958,-0.190980,0.024671,...,-0.035423,-0.006260,0.010149,-0.020688,0.017732,-0.064961,0.092112,0.007176,0.002569,-5.341681e-02
888,1.108195,-0.810913,-0.139839,-0.425952,-0.270007,-0.042247,0.098573,0.081613,0.144265,-0.076247,...,-0.020475,0.009303,-0.006730,-0.019834,-0.001333,0.010605,0.000588,0.002705,-0.002641,-4.859079e-17
889,-0.387596,1.572172,-0.061203,-0.131054,-0.116598,-0.008297,0.142011,0.054886,-0.155028,0.041220,...,-0.019032,-0.034302,-0.012208,0.047240,0.040511,-0.057019,0.055527,0.001545,0.017020,-9.469530e-02


In [16]:
feature_df

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,1.0,0.271174,0.125,0.000000,0.014151,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.472229,0.125,0.000000,0.139136,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.321438,0.000,0.000000,0.015469,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.434531,0.125,0.000000,0.103644,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.434531,0.000,0.000000,0.015713,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.5,0.334004,0.000,0.000000,0.025374,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
887,0.0,0.233476,0.000,0.000000,0.058556,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,1.0,0.216934,0.125,0.333333,0.045771,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,0.0,0.321438,0.000,0.000000,0.058556,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
combined_columns = sorted(list(set(feature_df.columns) | set(df_test.columns)))

missing_columns_in_df = [col for col in combined_columns if col not in feature_df.columns]
missing_columns_in_df_test = [col for col in combined_columns if col not in df_test.columns]

missing_cols_df = pd.DataFrame(0, index=feature_df.index, columns=missing_columns_in_df)
missing_cols_df_test = pd.DataFrame(0, index=df_test.index, columns=missing_columns_in_df_test)

feature_df = pd.concat([feature_df, missing_cols_df], axis=1)
df_test = pd.concat([df_test, missing_cols_df_test], axis=1)

feature_df = feature_df[sorted(feature_df.columns)]
df_test = df_test[feature_df.columns]

feature_df = feature_df.copy()
df_test = df_test.copy()

In [18]:
X, Y = df_pca, df['Survived']

len(X), len(Y)

(891, 891)

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

log_reg = LogisticRegression()

log_reg.fit(X_train,Y_train)

Y_pred = log_reg.predict(X_test)

Y_proba = log_reg.predict_proba(X_test)[:, 1]


In [20]:
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy}")

precision = precision_score(Y_test, Y_pred)
print(f"Precision: {precision}")

recall = recall_score(Y_test, Y_pred)
print(f"Recall: {recall}")

f1 = f1_score(Y_test, Y_pred)
print(f"F1 Score: {f1}")

roc_auc = roc_auc_score(Y_test, Y_proba)
print(f"ROC-AUC Score: {roc_auc}")

Accuracy: 0.8246268656716418
Precision: 0.7962962962962963
Recall: 0.7747747747747747
F1 Score: 0.7853881278538812
ROC-AUC Score: 0.8863831984851093


In [21]:
tuning_params = {
    'pca_transformer__pca__n_components': [5, 10, 15, 20, 25],  
    'logreg__C': np.logspace(-4, 4, 10),  
    'logreg__solver': ['liblinear', 'saga'],  
    'logreg__penalty': ['l1', 'l2'],  
    'logreg__max_iter': [5000, 10000, 20000]  
}

In [22]:
feature_df.columns

Index(['Age', 'Cabin_A10', 'Cabin_A11', 'Cabin_A14', 'Cabin_A16', 'Cabin_A18',
       'Cabin_A19', 'Cabin_A20', 'Cabin_A21', 'Cabin_A23',
       ...
       'Title_Major', 'Title_Master', 'Title_Miss', 'Title_Mlle', 'Title_Mme',
       'Title_Mr', 'Title_Mrs', 'Title_Ms', 'Title_Rev', 'Title_Sir'],
      dtype='object', length=216)

In [23]:
import re

patterns = [r'^Cabin', r'^Title']
compiled_patterns = [re.compile(pattern) for pattern in patterns]

def matches_any_pattern(column_name, compiled_patterns):
    return any(pattern.match(column_name) for pattern in compiled_patterns)

pca_columns = [col for col in feature_df.columns if matches_any_pattern(col, compiled_patterns)]

remaining_columns = set(feature_df.columns) - set(pca_columns)
print(remaining_columns)

{'Sex_female', 'Fare', 'Age', 'Parch', 'Sex_male', 'Embarked_S', 'Embarked_C', 'Embarked_nan', 'Pclass', 'Embarked_Q', 'SibSp'}


In [24]:
X = feature_df
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pca_transformer = ColumnTransformer(
    transformers=[
        ('pca', PCA(), pca_columns)  
    ],
    remainder="passthrough"  
)

pipeline = Pipeline(steps=[
    ('pca_transformer', pca_transformer),
    ('logreg', LogisticRegression()) 
])


clf = GridSearchCV(pipeline, tuning_params, cv=5, verbose=0)

best_model = clf.fit(X_train, Y_train)

Y_pred = best_model.predict(X_test)

Y_proba = best_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy}")

precision = precision_score(Y_test, Y_pred)
print(f"Precision: {precision}")

recall = recall_score(Y_test, Y_pred)
print(f"Recall: {recall}")

f1 = f1_score(Y_test, Y_pred)
print(f"F1 Score: {f1}")

roc_auc = roc_auc_score(Y_test, Y_proba)
print(f"ROC-AUC Score: {roc_auc}")


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Accuracy: 0.832089552238806
Precision: 0.8113207547169812
Recall: 0.7747747747747747
F1 Score: 0.792626728110599
ROC-AUC Score: 0.8917197452229298


In [25]:
best_model.best_estimator_

Pipeline(steps=[('pca_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pca', PCA(n_components=5),
                                                  ['Cabin_A10', 'Cabin_A11',
                                                   'Cabin_A14', 'Cabin_A16',
                                                   'Cabin_A18', 'Cabin_A19',
                                                   'Cabin_A20', 'Cabin_A21',
                                                   'Cabin_A23', 'Cabin_A24',
                                                   'Cabin_A26', 'Cabin_A29',
                                                   'Cabin_A31', 'Cabin_A32',
                                                   'Cabin_A34', 'Cabin_A36',
                                                   'Cabin_A5', 'Cabin_A6',
                                                   'Cabin_A7', 'Cabin_A9',
                                                   'Cabin_B10', 'Cabin_B101',
                                                   'Cabin_B102', 'Cabin_B11',
                                                   'Cabin_B18', 'Cabin_B19',
                                                   'Cabin_B20', 'Cabin_B22',
                                                   'Cabin_B24', 'Cabin_B26', ...])])),
                ('logreg',
                 LogisticRegression(C=21.54434690031882, max_iter=5000,
                                    solver='liblinear'))])

In [26]:
prediction = best_model.predict(df_test)

len(prediction)

418

In [27]:
passengerIds = [x for x in range(892, 1310)]

prediction_df = pd.DataFrame({'PassengerId': passengerIds, 'Survived': prediction})

prediction_df['Survived'] = prediction_df['Survived'].astype(int)

prediction_df.to_csv('titanic_pred.csv', index=False)